<a href="https://colab.research.google.com/github/somkiatth/git/blob/master/Copy_of_Recurrent_Neural_Networks_(RNNs)_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import InputLayer, Dense, Activation, Dropout, Conv1D, Embedding, SimpleRNN
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import spacy

nlp = spacy.load("en_core_web_sm") # โหลดโมเดลภาษาอังกฤษขนาดเล็กจาก spaCy เพื่อใช้ในการประมวลผลข้อความ เช่น การแยกคำ การแท็กไวยากรณ์ และการตรวจจับหน่วยข้อมูล

In [ ]:
# Download tweeter data from https://github.com/crwong/cs224u-project/blob/master/data/sentiment/training.1600000.processed.noemoticon.csv
# Use the raw GitHub URL for the CSV file
url = "https://raw.githubusercontent.com/crwong/cs224u-project/master/data/sentiment/training.1600000.processed.noemoticon.csv"

# Download the file
!wget {url} -O training.1600000.processed.noemoticon.csv

--2024-11-16 07:48:07--  https://raw.githubusercontent.com/crwong/cs224u-project/master/data/sentiment/training.1600000.processed.noemoticon.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2989873 (2.9M) [application/octet-stream]
Saving to: ‘training.1600000.processed.noemoticon.csv’

training.1600000.pr 100%[===================>]   2.85M  --.-KB/s    in 0.06s   

2024-11-16 07:48:07 (46.1 MB/s) - ‘training.1600000.processed.noemoticon.csv’ saved [2989873/2989873]



In [ ]:
#load the dataset
train=pd.read_csv("/content/training.1600000.processed.noemoticon.csv" , encoding= "latin-1")
Y_train = train[train.columns[0]]
X_train = train[train.columns[5]]

In [ ]:
train.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [ ]:
# split the data into test and train by train_test_split and then one-hot encoding for Y_train
from sklearn.model_selection import train_test_split
trainset1x, trainset2x, trainset1y, trainset2y = train_test_split(X_train.values, Y_train.values, test_size=0.02,random_state=42 )
trainset2y = pd.get_dummies(trainset2y)

In [ ]:
# function to remove stopwords
def stopwords(sentence):
    new = []                       # สร้าง list ว่างเพื่อเก็บคำที่ต้องการใช้
    sentence = nlp(sentence)       # แปลงข้อความเป็นวัตถุ Doc โดยใช้ spaCy
    for w in sentence:
        if (w.is_stop == False) & (w.pos_ != "PUNCT"):  # ตรวจสอบว่าคำไม่ใช่ stopword และไม่ใช่เครื่องหมายวรรคตอน
            new.append(w.string.strip())   # เพิ่มคำที่ผ่านการกรองลงใน list new
    c = " ".join(str(x) for x in new)      # รวมคำทั้งหมดที่อยู่ใน new เป็นประโยค
    return c                               # ส่งคืนประโยคที่ไม่มี stopwords และ punctuation

In [ ]:
# function to lemmatize the tweets
def lemmatize(sentence):
    sentence = nlp(sentence)      # แปลงข้อความเป็นวัตถุ Doc โดยใช้ spaCy
    str = ""                      # สร้างสตริงว่างสำหรับเก็บประโยคที่ผ่านการ lemmatize
    for w in sentence:            # ลูปผ่านแต่ละคำ (Token) ในประโยค
        str += " " + w.lemma_     # เพิ่ม lemma ของคำแต่ละคำลงใน str
    return nlp(str)               # ส่งคืนประโยคใหม่ที่ประกอบด้วย lemmatized tokens


In [ ]:
#loading the glove model
def loadGloveModel(gloveFile):
    print("Loading Glove Model")
    f = open(gloveFile, 'r')         # เปิดไฟล์ GloVe ที่ระบุ
    model = {}                       # สร้างดิกชันนารีเปล่าเพื่อเก็บคำและเวกเตอร์ฝังคำ
    for line in f:
        splitLine = line.split()      # แยกคำและค่าเวกเตอร์แต่ละบรรทัดด้วยช่องว่าง
        word = splitLine[0]           # คำแรกในบรรทัดคือคำ (word)
        embedding = [float(val) for val in splitLine[1:]]  # ค่าเวกเตอร์ฝังคำที่เหลือ
        model[word] = embedding       # เพิ่มคำและเวกเตอร์ในดิกชันนารี
    print("Done.", len(model), "words loaded!")  # แสดงข้อความเสร็จสิ้นและจำนวนคำที่โหลด
    return model                      # ส่งคืนดิกชันนารีที่เก็บคำและเวกเตอร์ฝังคำ


In [ ]:
!wget https://nlp.stanford.edu/data/glove.twitter.27B.zip

--2024-11-16 07:56:54--  https://nlp.stanford.edu/data/glove.twitter.27B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.twitter.27B.zip [following]
--2024-11-16 07:56:54--  https://downloads.cs.stanford.edu/nlp/data/glove.twitter.27B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1520408563 (1.4G) [application/zip]
Saving to: ‘glove.twitter.27B.zip’

glove.twitter.27B.z 100%[===================>]   1.42G  5.01MB/s    in 4m 45s  

2024-11-16 08:01:39 (5.09 MB/s) - ‘glove.twitter.27B.zip’ saved [1520408563/1520408563]



In [ ]:
!unzip glove.twitter.27B.zip glove.twitter.27B.200d.txt

Archive:  glove.twitter.27B.zip
  inflating: glove.twitter.27B.200d.txt  


In [ ]:
# save the glove model
# โหลดโมเดล GloVe ที่ใช้สำหรับการสร้างเวกเตอร์คำ โดยจะเก็บเวกเตอร์ในตัวแปร model
model=loadGloveModel("/content/glove.twitter.27B.200d.txt")

Loading Glove Model
Done. 1193514 words loaded!


In [ ]:
len(model)

1193514

In [ ]:
#vectorising the sentences สร้างฟังก์ชันที่ใช้เวกเตอร์คำจากโมเดล GloVe เพื่อสร้างเวกเตอร์ของประโยค
def sent_vectorizer(sent, model):
    sent_vec = np.zeros(200)         # สร้างเวกเตอร์ศูนย์ (vector of zeros) ขนาด 200 มิติ สำหรับเริ่มต้นเก็บค่าเวกเตอร์ของประโยค
    numw = 0                         # ตัวนับจำนวนคำที่มีอยู่ในโมเดล embedding
    for w in sent.split():           # แยกคำในประโยคโดยช่องว่างแล้ววนซ้ำแต่ละคำ
        try:
            sent_vec = np.add(sent_vec, model[str(w)])  # บวกเวกเตอร์ของคำแต่ละคำกับ sent_vec
            numw += 1                # นับจำนวนคำที่มีเวกเตอร์ในโมเดล
        except:
            pass                     # ถ้าไม่มีเวกเตอร์สำหรับคำใน model จะข้ามคำนี้ไป
    return sent_vec                  # ส่งคืนเวกเตอร์ประโยคที่คำนวณได้


In [ ]:
#obtain a clean vector ทำความสะอาดข้อมูลและสร้างเวกเตอร์สำหรับแต่ละทวีต จากนั้นแปลงเป็น NumPy array และปรับรูปทรงให้ตรงตามความต้องการ
cleanvector = []
for i in range(trainset2x.shape[0]):          # วนลูปผ่านข้อความทั้งหมดใน trainset2x
    document = trainset2x[i]                  # นำข้อความที่ตำแหน่ง i จาก trainset2x
    document = document.lower()               # แปลงข้อความให้เป็นตัวพิมพ์เล็ก
    document = lemmatize(document)            # นำข้อความไปแปลงเป็น lemmatize (ลดรูปคำ)
    document = str(document)                  # แปลงผลลัพธ์เป็นประเภท string (ข้อความ)
    cleanvector.append(sent_vectorizer(document, model))  # แปลงข้อความเป็นเวกเตอร์แล้วเพิ่มใน cleanvector


In [ ]:
trainset2x[0]

'2 more Exams '

In [ ]:
#Getting the input and output in proper shape
cleanvector = np.array(cleanvector)           # แปลง cleanvector เป็นอาเรย์ numpy
cleanvector = cleanvector.reshape(len(cleanvector), 200, 1)  # จัดรูปแบบอาเรย์ใหม่เป็น (จำนวนตัวอย่าง, 200, 1)

In [ ]:
#tokenizing the sequences ใช้ Tokenizer สำหรับการตัดลำดับข้อความและเติมคำที่หายไปให้มีความยาวเท่ากัน (padding) จากนั้นปรับรูปทรงของข้อมูล
tokenizer = Tokenizer(num_words=16000)                # สร้างตัวแปลงข้อความเป็นลำดับโดยใช้ 16,000 คำที่พบบ่อยที่สุด
tokenizer.fit_on_texts(trainset2x)                    # สร้างพจนานุกรมจากข้อความใน trainset2x
sequences = tokenizer.texts_to_sequences(trainset2x)  # แปลงข้อความใน trainset2x เป็นลำดับของหมายเลขแทนคำ
word_index = tokenizer.word_index                     # สร้างดิกชันนารีที่แมปคำแต่ละคำกับหมายเลขของมัน
print('Found %s unique tokens.' % len(word_index))    # แสดงจำนวนคำทั้งหมดใน word_index
data = pad_sequences(sequences, maxlen=15, padding="post")  # ปรับความยาวของ sequences ให้เท่ากันที่ 15 ตำแหน่ง
print(data.shape)                                     # แสดงรูปร่างของข้อมูล data หลังการ pad


Found 1873 unique tokens.
(400, 15)


In [ ]:
data=data.reshape(len(data),15,1)

In [ ]:
data.shape

(400, 15, 1)

In [ ]:
trainset2y.values[0]

array([False,  True])

In [ ]:
trainx, validx, trainy, validy = train_test_split(data, trainset2y.values, test_size=0.3,random_state=42 )

In [ ]:
trainx[0]

array([[1120],
       [ 477],
       [  91],
       [  70],
       [  66],
       [   4],
       [ 229],
       [  14],
       [ 357],
       [  11],
       [  31],
       [ 217],
       [   0],
       [   0],
       [   0]], dtype=int32)

In [ ]:
trainy[0]

array([False,  True])

In [ ]:
#calculate the number of words
nb_words=len(tokenizer.word_index)+1

In [ ]:
#obtain theembedding matrix
embedding_matrix = np.zeros((nb_words, 200)) # สร้างอาร์เรย์ที่มีขนาด (จำนวนคำ, 200)
for word, i in word_index.items():
    embedding_vector = model.get(word) # ดึงเวกเตอร์ฝังตัวที่เกี่ยวข้องกับคำปัจจุบันจาก GloVe model ที่โหลดไว้ก่อนหน้า
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
print('Null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0)) # นับจำนวนเวกเตอร์ใน embedding_matrix ที่มีค่าเป็น 0 (เวกเตอร์ที่ไม่ได้รับการกำหนดค่า)


Null word embeddings: 346


In [ ]:
embedding_matrix.shape

(1874, 200)

In [ ]:
len(tokenizer.word_index)

1873

In [ ]:
def modelbuild():
    model = Sequential(name='RNN')
    model.add(InputLayer(shape=(15,))) # ความยาว 15 ตำแหน่ง (15 timesteps)
    # input_dim=len(tokenizer.word_index) + 1: จำนวนคำที่ใช้ในโมเดล (รวมตำแหน่ง padding)
    # output_dim=200: ขนาดของเวกเตอร์ฝังตัวแต่ละคำคือ 200 มิติ
    # input_length=15: ความยาวของลำดับข้อมูลที่ป้อนเข้า
    # weights=[embedding_matrix]: ใช้ embedding_matrix ที่สร้างไว้เพื่อใช้เป็นเวกเตอร์ฝังตัวเริ่มต้น
    # trainable=False: หมายความว่าไม่อนุญาตให้ปรับค่าของเวกเตอร์ฝังตัวในระหว่างการฝึก (เพื่อรักษาค่าที่ได้จาก GloVe)
    model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=200, input_length=15, weights=[embedding_matrix], trainable=False))
    model.add(keras.layers.SimpleRNN(units=100, activation='relu', use_bias=True))
    model.add(keras.layers.Dense(units=1000, activation='sigmoid'))
    model.add(keras.layers.Dense(units=500, activation='relu'))
    model.add(keras.layers.Dense(units=2, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
trainx.shape, validx.shape, trainy.shape, validy.shape

((280, 15, 1), (120, 15, 1), (280, 2), (120, 2))

In [ ]:
#compiling the model
finalmodel = modelbuild()
finalmodel.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "RNN"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 15, 200)             │         374,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn (SimpleRNN)               │ (None, 100)                 │          30,100 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1000)                │         101,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 500)                 │         500,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 2)                   │           1,002 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,007,402 (3.84 MB)

 Trainable params: 632,602 (2.41 MB)

 Non-trainable params: 374,800 (1.43 MB)

In [ ]:
import time
start_time = time.time()
finalmodel.fit(trainx, trainy, epochs=30, batch_size=120,validation_data=(validx,validy))
end_time = time.time()
print(f"Training time: {end_time - start_time:.2f} seconds")

Epoch 1/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 375ms/step - accuracy: 0.5607 - loss: 1.7593 - val_accuracy: 0.5083 - val_loss: 1.5244
Epoch 2/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - accuracy: 0.5140 - loss: 1.6449 - val_accuracy: 0.5083 - val_loss: 1.1666
Epoch 3/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - accuracy: 0.5421 - loss: 0.9143 - val_accuracy: 0.4917 - val_loss: 1.1556
Epoch 4/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step - accuracy: 0.4766 - loss: 1.1740 - val_accuracy: 0.4917 - val_loss: 0.7062
Epoch 5/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - accuracy: 0.4815 - loss: 0.7329 - val_accuracy: 0.5083 - val_loss: 1.0511
Epoch 6/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - accuracy: 0.5296 - loss: 0.9830 - val_accuracy: 0.5083 - val_loss: 0.7139
Epoch 7/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - accuracy: 0.4981 - loss: 0.7056 - val_accuracy: 0.4917 - val_loss: 0.8309
Epoch 8/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - accuracy: 0.4725 - loss: 0.8172 - val_accuracy: 0.5167 - val_loss: 0.6845

In [ ]:
# Evaluate
score = finalmodel.evaluate(validx,validy, verbose=0)
print("Loss is", '{:.2f}'.format(score[0]*100))
print("Accuracy score is", '{:.2f}'.format(score[1]*100))

Loss is 123.51
Accuracy score is 66.67
